In [1]:
# required Python imports
import pandas as pd
import numpy as np
from types import SimpleNamespace

import ftir_funct as f
np.set_printoptions(suppress=True)

module FTIR v.2024.4.11 imported


## Generate database

In [2]:
f.explore_Euler_space(step=7, lower_bounds=(0, 0, 0), upper_bounds=(89, 89, 179))

array([[  0,   0,   0],
       [  0,   0,   7],
       [  0,   0,  14],
       ...,
       [ 84,  84, 161],
       [ 84,  84, 168],
       [ 84,  84, 175]])

In [3]:
# generate Euler angles each 7 degrees and store in the database
database = SimpleNamespace(euler=f.explore_Euler_space(step=7, lower_bounds=(0, 0, 0), upper_bounds=(89, 89, 179)))
database.euler.shape

(4394, 3)

In [4]:
dataset = pd.DataFrame(
    {
        "Euler1": database.euler[:, 0],
        "Euler2": database.euler[:, 1],
        "Euler3": database.euler[:, 2],
    }
)
dataset

,Euler1,Euler2,Euler3
0,0,0,0
1,0,0,7
2,0,0,14
3,0,0,21
4,0,0,28
...,...,...,...
4389,84,84,147
4390,84,84,154
4391,84,84,161
4392,84,84,168


In [5]:
# Transmission values for lambda 1987.29
Ta = 0.009882
Tb = 0.995815
Tc = 0.596951

# Generate a mesh of values defining the reference transmissión envelope
polar, azimuths = f.regular_S2_grid(n_squared=500)
T = f.Tvalues(trans=(Ta, Tb, Tc), azimuth=azimuths, polar=polar)
x, y, z = f.sph2cart(T, azimuths, polar)

In [6]:
# create a random generator
rgn = np.random.default_rng()

# set standard deviations for T and azimuths
std_T = 0.015
std_azimuths = 0.5

**2024-04-13 test**:  
std_T = 0.03  
std_azimuths = 0.5  

**2024-04-18 test**:  
std_T = 0.03  
std_azimuths = 0.25

**2024-04-19 test2**:  
std_T = 0.015  
std_azimuths = 0.5

In [7]:
# just a test
np.arange(0, 180, 9.0)

array([  0.,   9.,  18.,  27.,  36.,  45.,  54.,  63.,  72.,  81.,  90.,
        99., 108., 117., 126., 135., 144., 153., 162., 171.])

In [8]:
steps = (13.0, 11.5, 9.0, 7.5)  # in degrees
columns = ["n14", "n16", "n20", "n24"]

for i, step in enumerate(steps):
    # create database
    angles = np.arange(0, 180, step)
    T_vals = []
    azi_vals = []

    for euler in database.euler:
        # rotate
        x2, y2, z2 = f.rotate(coordinates=(x, y, z), euler_ang=euler)

        # extract XY intersection
        xy_vectors = f.extract_XY_section_fast2(x2, y2, z2)

        # get the indexes of specific angles
        indexes = f.find_nearest(xy_vectors["angles"], angles)

        # append values
        T_vals.append(xy_vectors.loc[indexes, ["T"]].T.values.tolist()[0])
        azi_vals.append(xy_vectors.loc[indexes, ["angles"]].T.values.tolist()[0])

    database.T_values = np.array(T_vals)
    database.azimuths = np.array(azi_vals)

    # add Gaussian noise to data
    database.T_noise = database.T_values + rgn.normal(0, std_T, database.T_values.shape)
    database.azi_noise = database.azimuths + rgn.normal(
        0, std_azimuths, database.azimuths.shape
    )

    # initialize variables
    no_noise = np.empty(database.euler.shape[0])
    noise = np.empty(database.euler.shape[0])

    for index, orientation in enumerate(database.euler):
        measures1 = np.column_stack(
            (
                database.T_values[index],
                database.azimuths[index],
                np.full_like(database.azimuths[index], 90),
            )
        )

        estimate1 = f.find_orientation(measurements=measures1, params=(Ta, Tb, Tc))
        no_noise[index] = f.calc_disorientation(orientation, estimate1.x)

        measures2 = np.column_stack(
            (
                database.T_noise[index],
                database.azi_noise[index],
                np.full_like(database.azimuths[index], 90),
            )
        )

        estimate2 = f.find_orientation(measurements=measures2, params=(Ta, Tb, Tc))
        noise[index] = f.calc_disorientation(orientation, estimate2.x)

    # add columns
    dataset[columns[i]] = no_noise
    dataset[columns[i] + "_noise"] = noise

    # clean database
    del database.T_values
    del database.azimuths
    del database.T_noise
    del database.azi_noise


In [9]:
# check
# print('Euler angles: ', database.euler[-1])
# print('T values: ', np.around(database.T_values[-1], 2))
# print('Azimuths: ', np.around(database.azimuths[-1], 1))
# print('T noise: ', np.around(database.T_noise[-1], 2))
# print('Azi_noise: ', np.around(database.azi_noise[-1], 1))

In [10]:
dataset

,Euler1,Euler2,Euler3,n14,n14_noise,n16,n16_noise,n20,n20_noise,n24,n24_noise
0,0,0,0,0.000,5.944,0.000,7.047,0.000,0.158,0.000,0.100
1,0,0,7,0.000,5.172,0.000,6.165,0.262,5.202,0.273,9.999
2,0,0,14,0.000,1.726,0.284,13.853,0.327,5.097,0.000,8.057
3,0,0,21,0.000,3.044,0.000,5.024,0.000,0.378,0.000,8.483
4,0,0,28,0.000,4.275,0.000,10.405,0.281,2.905,0.000,8.456
...,...,...,...,...,...,...,...,...,...,...,...
4389,84,84,147,0.008,3.408,0.013,3.328,0.011,2.764,0.008,2.150
4390,84,84,154,0.009,6.978,0.011,2.248,0.008,1.355,0.008,3.675
4391,84,84,161,0.012,4.625,0.008,4.154,0.009,7.105,0.010,6.404
4392,84,84,168,0.010,2.625,0.012,2.845,0.008,0.488,0.009,1.749


## Estimate

In [11]:
from datetime import date    
today = date.today().isoformat()

In [12]:
dataset.to_csv(today + '_step7_lamb1987_noise_datapoints2.csv', index=False)

In [13]:
# quick statistical overview
dataset[['n14', 'n14_noise', 'n16', 'n16_noise', 'n20', 'n20_noise', 'n24', 'n24_noise']].describe()

,n14,n14_noise,n16,n16_noise,n20,n20_noise,n24,n24_noise
count,4394.000000,4394.000000,4394.000000,4394.000000,4394.000000,4394.000000,4394.000000,4394.000000
mean,0.030134,3.226496,0.027723,3.156962,0.029225,2.871230,0.027764,2.757843
std,0.148653,2.524908,0.125815,2.520365,0.177930,2.331134,0.145089,2.349803
min,0.000000,0.006000,0.000000,0.005000,0.000000,0.002000,0.000000,0.004000
25%,0.003000,1.417500,0.002000,1.374250,0.002000,1.223250,0.002000,1.100250
50%,0.005000,2.411500,0.004000,2.324000,0.004000,2.080500,0.004000,1.892500
75%,0.011000,4.363250,0.010000,4.244000,0.010000,3.884250,0.009000,3.754000
max,6.121000,17.574000,5.366000,18.107000,8.680000,17.846000,5.879000,17.811000
